In [1]:
import pandas as pd
from pandas import option_context
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(transform_output = "pandas")

#Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import xgboost as xgb

#MissingIndicator, Imputer and Pipeline
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.metrics import make_scorer, precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, roc_auc_score

In [2]:
RAND_SEED = 7
SPLIT = 0.4

## Read ground truth dataset

In [3]:
ground_truth = pd.read_csv('../data/total_contributors_with_dataset.csv').set_index('contributor')

## Read the features and display

In [4]:
df_features_read = pd.read_csv('../data/features0305.csv',index_col=0).sort_index()

In [5]:
df_features = (
               df_features_read
               .merge(ground_truth.drop('dataset',axis=1), left_index=True, right_index=True)
               .assign(bot = lambda d: np.where(d.acc_type_y == "human", 0, 1))
               .drop(['acc_type_x', 'acc_type_y'],axis=1)
               # .assign(contributor = lambda d: d.index.str.replace('[bot]','',regex=False))
               # .set_index('contributor')
               .sort_index()
              )
with option_context('display.max_column',None):
    display(df_features.query('bot==1'))

,feat_num_activities,feat_num_activity_type,feat_repos,feat_owner,feat_ratio_owner_repo,consecutive_activity_time_mean,consecutive_activity_time_50%,consecutive_activity_time_std,consecutive_activity_time_gini,consecutive_activity_time_IQR,activity_per_repo_mean,activity_per_repo_50%,activity_per_repo_std,activity_per_repo_gini,activity_per_repo_IQR,activity_type_per_repo_mean,activity_type_per_repo_50%,activity_type_per_repo_std,activity_type_per_repo_gini,activity_type_per_repo_IQR,continuous_activity_repo_mean,continuous_activity_repo_50%,continuous_activity_repo_std,continuous_activity_repo_gini,continuous_activity_repo_IQR,time_spent_in_repo_mean,time_spent_in_repo_50%,time_spent_in_repo_std,time_spent_in_repo_gini,time_spent_in_repo_IQR,time_to_switch_repo_mean,time_to_switch_repo_50%,time_to_switch_repo_std,time_to_switch_repo_gini,time_to_switch_repo_IQR,time_to_switch_act_type_mean,time_to_switch_act_type_50%,time_to_switch_act_type_std,time_to_switch_act_type_gini,time_to_switch_act_type_IQR,activity_per_act_type_mean,activity_per_act_type_50%,activity_per_act_type_std,activity_per_act_type_gini,activity_per_act_type_IQR,bot
contributor,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1gtm,299,3,23,4,0.174,0.081,0.000,0.679,0.970,0.003,13.000,8.0,13.655,0.490,10.00,1.957,2.0,0.367,0.062,0.0,6.795,3.0,9.849,0.567,4.25,0.206,0.001,1.275,0.956,0.010,0.352,0.017,1.212,0.897,0.045,0.017,0.000,0.191,0.950,0.001,99.667,146.0,85.500,0.337,75.50,1
24emebot,279,1,1,1,1.000,7.589,0.584,15.767,0.786,6.146,279.000,279.0,0.000,0.000,0.00,1.000,1.0,0.000,0.000,0.0,279.000,279.0,0.000,0.000,0.00,2109.750,2109.750,0.000,0.000,0.000,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,0.000,NaN,279.000,279.0,0.000,0.000,0.00,1
47erbot,201,7,12,3,0.250,1.930,0.003,9.800,0.960,0.012,16.750,12.0,15.058,0.392,8.50,4.667,5.0,1.231,0.095,0.0,2.871,3.0,1.710,0.275,1.00,1.237,0.000,7.341,0.965,0.004,4.339,0.012,14.939,0.927,0.020,2.013,0.001,10.149,0.958,0.010,28.714,31.0,12.945,0.215,6.50,1
5imon-bot,56,1,3,1,0.333,36.690,0.311,76.716,0.813,26.075,18.667,8.0,19.348,0.405,17.00,1.000,1.0,0.000,0.000,0.0,4.308,1.0,8.430,0.629,2.00,33.451,0.000,62.265,0.519,45.408,131.922,114.406,115.178,0.456,134.859,NaN,NaN,0.000,0.000,NaN,56.000,56.0,0.000,0.000,0.00,1
9cibot,27,2,1,1,1.000,38.672,0.097,112.949,0.864,0.215,27.000,27.0,0.000,0.000,0.00,2.000,2.0,0.000,0.000,0.0,27.000,27.0,0.000,0.000,0.00,1005.467,1005.467,0.000,0.000,0.000,NaN,NaN,0.000,0.000,NaN,10.420,0.048,41.244,0.884,0.102,13.500,13.5,7.778,0.204,5.50,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zksync-admin-bot2,51,4,5,1,0.200,41.947,0.560,97.237,0.820,39.724,10.200,3.0,11.584,0.518,14.00,2.400,2.0,0.548,0.100,1.0,3.923,3.0,3.201,0.395,3.00,34.610,0.196,94.986,0.827,12.673,137.286,86.892,159.089,0.535,95.312,43.350,0.095,108.175,0.848,13.403,12.750,13.5,5.909,0.221,5.75,1
zorro-bot[bot],85,3,1,1,1.000,25.382,11.938,39.925,0.660,26.623,85.000,85.0,0.000,0.000,0.00,3.000,3.0,0.000,0.000,0.0,85.000,85.0,0.000,0.000,0.00,2132.089,2132.089,0.000,0.000,0.000,NaN,NaN,0.000,0.000,NaN,30.598,17.293,41.522,0.584,30.556,28.333,8.0,38.734,0.541,34.50,1
zube[bot],43,3,7,5,0.714,12.025,1.843,21.076,0.746,12.083,6.143,1.0,7.034,0.545,9.50,1.429,1.0,0.535,0.171,1.0,2.389,1.0,2.429,0.443,1.00,9.460,0.000,26.561,0.717,0.015,19.693,6.962,28.264,0.655,14.387,12.800,6.231,21.420,0.698,6.705,14.333,4.0,19.655,0.543,17.50,1


In [6]:
df_features.columns[df_features.isna().any()]

Index(['time_to_switch_repo_mean', 'time_to_switch_repo_50%',
       'time_to_switch_repo_IQR', 'time_to_switch_act_type_mean',
       'time_to_switch_act_type_50%', 'time_to_switch_act_type_IQR'],
      dtype='object')

In [7]:
df_features.bot.value_counts()

0    1115
1    1035
Name: bot, dtype: int64

## Split data to train and test

In [8]:
def data_split(df):
    '''
    args: df - DataFrame; contributor behavioural features based on their activities
    
    return: x_train - DataFrame; features for training the model
            y_train - DataFrame; labels that represent bot/human for trianing the model
            x_test - DataFrame; features for testing the model
            y_test - DataFrame; labels that represent bot/human for testing the model
    
    method: invoke train_test_split to split the data and the labels accordingly
    '''
    
    x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,:-1], 
                                                        df.iloc[:,-1:], 
                                                        test_size=SPLIT, 
                                                        random_state=RAND_SEED,
                                                        stratify=df.iloc[:,-1:])
    
    return(x_train, x_test, y_train, y_test)

In [9]:
x_train, x_test, y_train, y_test = data_split(df_features)

In [10]:
x_train.to_csv('../data/train_feat.csv')
y_train.to_csv('../data/train_label.csv')
x_test.to_csv('../data/test_feat.csv')
y_test.to_csv('../data/test_label.csv')

In [11]:
total = df_features.shape[0]
bot = df_features.query('bot == 1').shape[0]
hum = df_features.query('bot == 0').shape[0]

print(f'original data: \n   total number of contributors: {total} \n   bot contributors: {bot} \n   human contributors: {hum}')

tr_total = y_train.shape[0]
tr_bot = y_train.query('bot == 1').shape[0]
tr_hum = y_train.query('bot == 0').shape[0]

print(f'\ntrain data: \n   total number of contributors: {tr_total} \n   bot contributors: {tr_bot} \n   human contributors: {tr_hum}')

print(f'\ntraining proportion: \n   total number of contributors: {tr_total/total} \n   bot contributors: {tr_bot/bot} \n   human contributors: {tr_hum/hum}')

te_total = y_test.shape[0]
te_bot = y_test.query('bot == 1').shape[0]
te_hum = y_test.query('bot == 0').shape[0]

print(f'\ntest data: \n   total number of contributors: {te_total} \n   bot contributors: {te_bot} \n   human contributors: {te_hum}')

original data: 
   total number of contributors: 2150 
   bot contributors: 1035 
   human contributors: 1115

train data: 
   total number of contributors: 1290 
   bot contributors: 621 
   human contributors: 669

training proportion: 
   total number of contributors: 0.6 
   bot contributors: 0.6 
   human contributors: 0.6

test data: 
   total number of contributors: 860 
   bot contributors: 414 
   human contributors: 446


In [12]:
def botrecall(y_true, y_pred):   
    return precision_recall_fscore_support(y_true,y_pred,zero_division=0.0)[1][1]
bot_recall = make_scorer(botrecall, greater_is_better=True)

def humanrecall(y_true, y_pred):   
    return precision_recall_fscore_support(y_true,y_pred,zero_division=0.0)[1][0]
human_recall = make_scorer(humanrecall, greater_is_better=True)

def botprecision(y_true, y_pred):   
    return precision_recall_fscore_support(y_true,y_pred,zero_division=0.0)[0][1]
bot_precision = make_scorer(botprecision, greater_is_better=True)

def humanprecision(y_true, y_pred):   
    return precision_recall_fscore_support(y_true,y_pred,zero_division=0.0)[0][0]
human_precision = make_scorer(humanprecision, greater_is_better=True)

def botfscore(y_true, y_pred):   
    return precision_recall_fscore_support(y_true,y_pred,zero_division=0.0)[2][1]
bot_fscore = make_scorer(botfscore, greater_is_better=True)

def humanfscore(y_true, y_pred):
    return precision_recall_fscore_support(y_true,y_pred,zero_division=0.0)[2][0]
human_fscore = make_scorer(humanfscore, greater_is_better=True)

def wpscore(y_true, y_pred):   
    return precision_recall_fscore_support(y_true,y_pred,average='weighted',zero_division=0.0)[0]
wprecision_score = make_scorer(wpscore, greater_is_better=True)

def wrscore(y_true, y_pred):   
    return precision_recall_fscore_support(y_true,y_pred,average='weighted',zero_division=0.0)[1]
wrecall_score = make_scorer(wrscore, greater_is_better=True)

def wfscore(y_true, y_pred):   
    return precision_recall_fscore_support(y_true,y_pred,average='weighted',zero_division=0.0)[2]
wf1_score = make_scorer(wfscore, greater_is_better=True)

def wrocauc(y_true, y_pred):
    return roc_auc_score(y_true, y_pred, average='weighted')
wrocauc_score = make_scorer(wrocauc, greater_is_better=True)

In [13]:
'''
Define BIN approach
'''
def BIN(data):
    terms=['bot']
    preds=(
           data
           .reset_index()
           .assign(pred = lambda d: np.where(d.contributor.str.contains('|'.join(terms)), 1, 0))
           .pred.to_numpy()
    )
    return(preds)

In [14]:
x_train_bin = x_train.assign(contributor=lambda d: d.index.str.replace('[bot]','',regex=False)).set_index('contributor')
y_train_bin = y_train.assign(contributor=lambda d: d.index.str.replace('[bot]','',regex=False)).set_index('contributor')

In [15]:
'''
Make statified split so that this model can use the same train and test dataset that are going to be used in grid-search cross-validation
'''
sss = StratifiedShuffleSplit(n_splits = 10, test_size=0.1, random_state=RAND_SEED)
split_index_train={}
split_index_test={}
for i, (train_index, test_index) in enumerate(sss.split(x_train_bin, y_train_bin['bot'].values)):
    split_index_train[f'fold{i}'] = train_index
    split_index_test[f'fold{i}'] = test_index

In [16]:
'''
Get the performance metric at two levels
1. For each split
2. Mean scores obtained for all splits considered together
'''
performance = pd.DataFrame() # for scores obtained in each split
# to calculate mean
bin_bot_f1 = []
bin_human_f1 = []
bin_f1_weighted = []
bin_bot_precision = []
bin_bot_recall = []
bin_human_precision = []
bin_human_recall = []
bin_precision = []
bin_recall = []
bin_roc_auc = []

for i in range(len(split_index_train)):
    train_sample_features = x_train_bin.iloc[split_index_train[f'fold{i}']]
    train_sample_labels = y_train_bin.iloc[split_index_train[f'fold{i}']]['bot'].values
    preds_sample = BIN(train_sample_features)
    split_scores = {f'split{i}_test_bot_f1':[botfscore(train_sample_labels, preds_sample)],
                    f'split{i}_test_human_f1':[humanfscore(train_sample_labels, preds_sample)],
                    f'split{i}_test_f1_weighted':[wfscore(train_sample_labels, preds_sample)],
                    f'split{i}_test_bot_precision':[botprecision(train_sample_labels, preds_sample)],
                    f'split{i}_test_bot_recall':[botrecall(train_sample_labels, preds_sample)],
                    f'split{i}_test_human_precision':[humanprecision(train_sample_labels, preds_sample)],
                    f'split{i}_test_human_recall':[humanrecall(train_sample_labels, preds_sample)],
                    f'split{i}_test_precision':[wpscore(train_sample_labels, preds_sample)],
                    f'split{i}_test_recall':[wrscore(train_sample_labels, preds_sample)],
                    f'split{i}_test_roc_auc':[wrocauc(train_sample_labels, preds_sample)]}
    
    bin_bot_f1.extend([botfscore(train_sample_labels, preds_sample)])
    bin_human_f1.extend([humanfscore(train_sample_labels, preds_sample)])
    bin_f1_weighted.extend([wfscore(train_sample_labels, preds_sample)])
    bin_bot_precision.extend([botprecision(train_sample_labels, preds_sample)])
    bin_bot_recall.extend([botrecall(train_sample_labels, preds_sample)])
    bin_human_precision.extend([humanprecision(train_sample_labels, preds_sample)])
    bin_human_recall.extend([humanrecall(train_sample_labels, preds_sample)])
    bin_precision.extend([wpscore(train_sample_labels, preds_sample)])
    bin_recall.extend([wrscore(train_sample_labels, preds_sample)])
    bin_roc_auc.extend([wrocauc(train_sample_labels, preds_sample)])
    
    performance = pd.concat([performance, pd.DataFrame.from_dict(split_scores).T])
    # display(train_sample_features)
    # display(train_sample_labels)
    # display(preds_sample)
performance = performance.T
display(performance)
mean_std_performance={'method':'BIN',
                      'mean_test_bot_f1':[np.mean(bin_bot_f1)],
                      'std_test_bot_f1':[np.std(bin_bot_f1)],
                      'mean_test_human_f1':[np.mean(bin_human_f1)],
                      'std_test_human_f1':[np.std(bin_human_f1)],
                      'mean_test_f1_weighted':[np.mean(bin_f1_weighted)],
                      'std_test_f1_weighted':[np.std(bin_f1_weighted)],
                      'mean_test_bot_precision':[np.mean(bin_bot_precision)],
                      'std_test_bot_precision':[np.std(bin_bot_precision)],
                      'mean_test_human_precision':[np.mean(bin_human_precision)],
                      'std_test_human_precision':[np.std(bin_human_precision)],
                      'mean_test_bot_recall':[np.mean(bin_bot_recall)],
                      'std_test_bot_recall':[np.std(bin_bot_recall)],
                      'mean_test_human_recall':[np.mean(bin_human_recall)],
                      'std_test_human_recall':[np.std(bin_human_recall)],
                      'mean_test_precision':[np.mean(bin_precision)],
                      'std_test_precision':[np.std(bin_precision)],
                      'mean_test_recall':[np.mean(bin_recall)],
                      'std_test_recall':[np.std(bin_recall)],
                      'mean_test_auc_roc':[np.mean(bin_roc_auc)],
                      'std_test_auc_roc':[np.std(bin_roc_auc)]
                     }
BIN_performance=pd.DataFrame.from_dict(mean_std_performance)
display(BIN_performance)
# pd.DataFrame.from_records(performance)

,split0_test_bot_f1,split0_test_human_f1,split0_test_f1_weighted,split0_test_bot_precision,split0_test_bot_recall,split0_test_human_precision,split0_test_human_recall,split0_test_precision,split0_test_recall,split0_test_roc_auc,...,split9_test_bot_f1,split9_test_human_f1,split9_test_f1_weighted,split9_test_bot_precision,split9_test_bot_recall,split9_test_human_precision,split9_test_human_recall,split9_test_precision,split9_test_recall,split9_test_roc_auc
0,0.7098,0.760031,0.735846,0.75813,0.667263,0.721973,0.802326,0.739382,0.737295,0.734794,...,0.720686,0.769835,0.746171,0.771429,0.676208,0.730253,0.813953,0.750078,0.747631,0.745081


,method,mean_test_bot_f1,std_test_bot_f1,mean_test_human_f1,std_test_human_f1,mean_test_f1_weighted,std_test_f1_weighted,mean_test_bot_precision,std_test_bot_precision,mean_test_human_precision,...,mean_test_bot_recall,std_test_bot_recall,mean_test_human_recall,std_test_human_recall,mean_test_precision,std_test_precision,mean_test_recall,std_test_recall,mean_test_auc_roc,std_test_auc_roc
0,BIN,0.713773,0.004368,0.764777,0.004568,0.74022,0.00436,0.765412,0.007047,0.724649,...,0.668694,0.004511,0.809635,0.007324,0.744275,0.004752,0.741774,0.004422,0.739164,0.004349


## Model search

### Pipeline construction

In [17]:
def make_nested_imputers(df):
    '''
    args: df - DataFrame; contributor behavioural features
    
    returns: preprocessor - ColumnTransformer that specifies all the indicator, imputers
    
    method: Define all the individual indicator and imputers and give it to the column transformer 
            in the form of list of tuples. Each tuple is of the form 
            ('transformer name', 'transformer', 'column names')  
    '''
    
    #indicator and imputer
    imputers = {'nan_indicator': MissingIndicator(missing_values=np.nan, features='all'),
                'nan_imputer': SimpleImputer(missing_values=np.nan, strategy='median', 
                                             keep_empty_features = True)
               }
    
    #names for indicator and imputers
    imputer_names = {'nan_indicator': 'indicate_nan',
                     'nan_imputer': 'impute_nan'
                    }
    
    #group of columns to be used in indicator and imputers

    nan_col_names = df.columns[df.isna().any()].to_list()
    
    nested_transformers = [(imputer_names[imputer], imputers[imputer], nan_col_names) 
                           for imputer in imputers]
    preprocessor = ColumnTransformer(transformers=nested_transformers,
                                     remainder='passthrough',
                                     verbose_feature_names_out=False)
    return(preprocessor)

In [18]:
def make_custom_pipeline(df, clf):
    '''
    args: df - DataFrame of features
          clf - classifier to be used as an estimator
    
    returns: pipe - Pipeline of transformers to be applied on the data
    
    method: invoke make_nested_imputers for getting all the transformations that has to applied on 
            the data and build the pipeline with nested transformations and classifier
    '''
    
    preprocessor = make_nested_imputers(df)
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('clf', clf)
    ])
    
    return(pipe)

### Grid search

In [19]:
def perform_grid_search(training_data_features, training_data_labels, clf, param_grid, clf_method):
    '''
    args: training_data_features - DataFrame; to train/cross-validate the model
          training_data_labels - array; array of labels (bot/human) for each row of features
          clf - method; classifier
          param_grid - dict; classifier's parameters for performing grid search
          clf_method - str; classifier name as a string
    
    invoke: make_custom_pipeline(...)
    
    returns: df_model_performance - DataFrame; performance of the model for the set of 
             parameters in grid search
    
    method: construct the pipeline with the given classifier as estimator
            perform grid search and store all the performance metrics for all combinations in a dataframe
    '''
    pipe = make_custom_pipeline(training_data_features, clf)
    grid_search = GridSearchCV(pipe, param_grid=param_grid,
                               cv=StratifiedShuffleSplit(n_splits = 10, random_state=RAND_SEED),
                               scoring={'bot_f1':bot_fscore,
                                        'human_f1':human_fscore,
                                        'f1_micro':'f1_micro',
                                        'f1_macro':'f1_macro',
                                        'f1_weighted':'f1_weighted',
                                        'bot_precision':bot_precision,
                                        'human_precision':human_precision,
                                        'precision':wprecision_score,
                                        'bot_recall':bot_recall,
                                        'human_recall':human_recall,
                                        'recall':wrecall_score,
                                        'roc_auc':'roc_auc'
                                  },refit=False, n_jobs=-1)
    display(grid_search)
    grid_search.fit(training_data_features,training_data_labels)
    # Create a data frame of resulting classifiers with precision scores
    df_model_performance = (
                         pd.DataFrame()
                         .from_dict(grid_search.cv_results_)
                         .assign(method=clf_method)
                        )
    
    return(df_model_performance)

### Classifiers and parameters

In [20]:
# parameters for GaussianNB = 1 model
gnb_param_grid = {}

# parameters for DecisionTreeClassifier 10*2*1 = 20 models
dtc_param_grid = {'clf__max_depth': [3, 4, 6, 8, 10, 12, 14, 16, 18, None],
                  'clf__criterion': ['gini','entropy'],
                  'clf__class_weight': ['balanced']
                 }

# parameters for RandomForestClassifier 10*9*3*1*3 = 810 models
rfc_param_grid = {'clf__max_depth': [3, 4, 6, 8, 10, 12, 14, 16, 18, None],
                  'clf__n_estimators': [25, 50, 75, 100, 125, 150, 175, 200, 225],
                  'clf__criterion': ['gini','entropy','log_loss'],
                  'clf__class_weight': ['balanced'],
                  'clf__max_features': ['sqrt','log2',None]
                 }

# parameters for GradientBoostingClassifier 2*5*9*2*3*10 = 5,400 models
gbc_param_grid = {'clf__loss': ['log_loss','exponential'],
                  'clf__learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
                  'clf__n_estimators': [25, 50, 75, 100, 125, 150, 175, 200, 225],
                  'clf__criterion': ['friedman_mse','squared_error'],
                  'clf__max_features': ['sqrt','log2',None],
                  'clf__max_depth': [3, 4, 6, 8, 10, 12, 14, 16, 18, None]
                 }

# parameters for SupportVectorMachineClassifier (SVC) 3*3*2*2*1 = 36 models
svc_param_grid = {'clf__kernel': ['poly','rbf','sigmoid'],
                  'clf__degree': [2, 3, 4],# 5, 6], # 5 and 6 are taking too much time
                  'clf__C': [0.1, 0.5],# 1, 2], #1.5 not converging, 1 and 2 are taking too much time
                  'clf__gamma': ['scale', 'auto'],
                  'clf__class_weight': ['balanced']
                 }

# parameters for LinearDiscriminantAnalysis 1*2 = 2 models
lda_param_grid = {'clf__solver': ['svd','lsqr']}

# parameters for xgboost with gbtree booster 9*10*5*1*3*1 = 1,350 models
xgb_tree_param_grid = {'clf__n_estimators': [25, 50, 75, 100, 125, 150, 175, 200, 225],
                       'clf__max_depth': [3, 4, 6, 8, 10, 12, 14, 16, 18, None],
                       'clf__learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
                       'clf__booster': ['gbtree'],
                       'clf__eval_metric': ['error', 'logloss', 'mae'],
                       'clf__objective': ['binary:logistic']
                      }

# parameters for xgboost with dart booster 9*10*5*1*3*1*2*2 = 5,400 models
xgb_dart_param_grid = {'clf__n_estimators': [25, 50, 75, 100, 125, 150, 175, 200, 225],
                       'clf__max_depth': [3, 4, 6, 8, 10, 12, 14, 16, 18, None],
                       'clf__learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
                       'clf__booster': ['dart'],
                       'clf__eval_metric': ['error', 'logloss', 'mae'],
                       'clf__objective': ['binary:logistic'],
                       'clf__sample_type': ['uniform','weighted'],
                       'clf__normalize_type': ['tree', 'forest']
                      }

# parameters for xgboost with linear booster 1*1*2 = 2 models
xgb_linear_param_grid = {'clf__booster': ['gblinear'],
                         'clf__updater': ['shotgun'],
                         'clf__feature_selector': ['cyclic', 'shuffle']
                        }

In [21]:
num_model = 1+20+810+5400+36+2+1350+5400+2
print(f'total number of models = {num_model}')

total number of models = 13021


In [22]:
'''
Classifiers and their parameters
'''
classifiers = {
               'GaussianNB':GaussianNB(),
               'DecisionTreeClassifier':DecisionTreeClassifier(random_state=RAND_SEED),
               'RandomForestClassifier':RandomForestClassifier(random_state=RAND_SEED),
               'GradientBoostingClassifier':GradientBoostingClassifier(random_state=RAND_SEED),
               'SVC':SVC(random_state=RAND_SEED),
               'LinearDiscriminantAnalysis':LinearDiscriminantAnalysis(),
               'XGBClassifier_tree':xgb.XGBClassifier(random_state=RAND_SEED),
               'XGBClassifier_dart':xgb.XGBClassifier(random_state=RAND_SEED),
               'XGBClassifier_linear':xgb.XGBClassifier(random_state=RAND_SEED)
              }
classifiers_params ={
               'GaussianNB':gnb_param_grid,
               'DecisionTreeClassifier':dtc_param_grid,
               'RandomForestClassifier':rfc_param_grid,
               'GradientBoostingClassifier':gbc_param_grid,
               'SVC':svc_param_grid,
               'LinearDiscriminantAnalysis':lda_param_grid,
               'XGBClassifier_tree':xgb_tree_param_grid,
               'XGBClassifier_dart':xgb_dart_param_grid,
               'XGBClassifier_linear':xgb_linear_param_grid
}

In [23]:
df_models_performance_all = pd.DataFrame()

for clf in tqdm(classifiers.keys()):
    df_model_performance = perform_grid_search(x_train, 
                                               y_train['bot'].values, 
                                               classifiers[clf], 
                                               classifiers_params[clf],
                                               clf)
    df_models_performance_all = pd.concat([df_models_performance_all, df_model_performance])

  0%|                                                                                                                                                                 | 0/9 [00:00<?, ?it/s]

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=7, test_size=None,
            train_size=None),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('indicate_nan',
                                                                         MissingIndicator(features='all'),
                                                                         ['time_to_switch_repo_mean',
                                                                          'time_to_switch_repo_50%',
                                                                          'time_to_switch_repo_IQR',
                                                                          'time_to_switch_act_type_...
                      'f1_weighted': 'f1_weighted',
                      'human_f1': make_scorer(humanfscore, response_method='predict'),
                      'human_precision': make_scorer(humanprecision, response_method='predict'),
                      'human_recall': make_scorer(humanrecall, response_method='predict'),
                      'precision': make_scorer(wpscore, response_method='predict'),
                      'recall': make_scorer(wrscore, response_method='predict'),
                      'roc_auc': 'roc_auc'})

 11%|█████████████████                                                                                                                                        | 1/9 [00:01<00:14,  1.75s/it]

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=7, test_size=None,
            train_size=None),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('indicate_nan',
                                                                         MissingIndicator(features='all'),
                                                                         ['time_to_switch_repo_mean',
                                                                          'time_to_switch_repo_50%',
                                                                          'time_to_switch_repo_IQR',
                                                                          'time_to_switch_act_type_...
                      'f1_weighted': 'f1_weighted',
                      'human_f1': make_scorer(humanfscore, response_method='predict'),
                      'human_precision': make_scorer(humanprecision, response_method='predict'),
                      'human_recall': make_scorer(humanrecall, response_method='predict'),
                      'precision': make_scorer(wpscore, response_method='predict'),
                      'recall': make_scorer(wrscore, response_method='predict'),
                      'roc_auc': 'roc_auc'})

 22%|██████████████████████████████████                                                                                                                       | 2/9 [00:04<00:15,  2.26s/it]

...

 78%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                | 7/9 [7:40:10<2:01:33, 3646.50s/it]

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=7, test_size=None,
            train_size=None),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('indicate_nan',
                                                                         MissingIndicator(features='all'),
                                                                         ['time_to_switch_repo_mean',
                                                                          'time_to_switch_repo_50%',
                                                                          'time_to_switch_repo_IQR',
                                                                          'time_to_switch_act_type_...
                      'f1_weighted': 'f1_weighted',
                      'human_f1': make_scorer(humanfscore, response_method='predict'),
                      'human_precision': make_scorer(humanprecision, response_method='predict'),
                      'human_recall': make_scorer(humanrecall, response_method='predict'),
                      'precision': make_scorer(wpscore, response_method='predict'),
                      'recall': make_scorer(wrscore, response_method='predict'),
                      'roc_auc': 'roc_auc'})

 89%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                | 8/9 [22:47:32<5:30:17, 19817.25s/it]

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=7, test_size=None,
            train_size=None),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('indicate_nan',
                                                                         MissingIndicator(features='all'),
                                                                         ['time_to_switch_repo_mean',
                                                                          'time_to_switch_repo_50%',
                                                                          'time_to_switch_repo_IQR',
                                                                          'time_to_switch_act_type_...
                      'f1_weighted': 'f1_weighted',
                      'human_f1': make_scorer(humanfscore, response_method='predict'),
                      'human_precision': make_scorer(humanprecision, response_method='predict'),
                      'human_recall': make_scorer(humanrecall, response_method='predict'),
                      'precision': make_scorer(wpscore, response_method='predict'),
                      'recall': make_scorer(wrscore, response_method='predict'),
                      'roc_auc': 'roc_auc'})

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [22:47:33<00:00, 9117.02s/it]


In [24]:
with option_context('display.max_column',None):
    display(df_models_performance_all)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_bot_f1,split1_test_bot_f1,split2_test_bot_f1,split3_test_bot_f1,split4_test_bot_f1,split5_test_bot_f1,split6_test_bot_f1,split7_test_bot_f1,split8_test_bot_f1,split9_test_bot_f1,mean_test_bot_f1,std_test_bot_f1,rank_test_bot_f1,split0_test_human_f1,split1_test_human_f1,split2_test_human_f1,split3_test_human_f1,split4_test_human_f1,split5_test_human_f1,split6_test_human_f1,split7_test_human_f1,split8_test_human_f1,split9_test_human_f1,mean_test_human_f1,std_test_human_f1,rank_test_human_f1,split0_test_f1_micro,split1_test_f1_micro,split2_test_f1_micro,split3_test_f1_micro,split4_test_f1_micro,split5_test_f1_micro,split6_test_f1_micro,split7_test_f1_micro,split8_test_f1_micro,split9_test_f1_micro,mean_test_f1_micro,std_test_f1_micro,rank_test_f1_micro,split0_test_f1_macro,split1_test_f1_macro,split2_test_f1_macro,split3_test_f1_macro,split4_test_f1_macro,split5_test_f1_macro,split6_test_f1_macro,split7_test_f1_macro,split8_test_f1_macro,split9_test_f1_macro,mean_test_f1_macro,std_test_f1_macro,rank_test_f1_macro,split0_test_f1_weighted,split1_test_f1_weighted,split2_test_f1_weighted,split3_test_f1_weighted,split4_test_f1_weighted,split5_test_f1_weighted,split6_test_f1_weighted,split7_test_f1_weighted,split8_test_f1_weighted,split9_test_f1_weighted,mean_test_f1_weighted,std_test_f1_weighted,rank_test_f1_weighted,split0_test_bot_precision,split1_test_bot_precision,split2_test_bot_precision,split3_test_bot_precision,split4_test_bot_precision,split5_test_bot_precision,split6_test_bot_precision,split7_test_bot_precision,split8_test_bot_precision,split9_test_bot_precision,mean_test_bot_precision,std_test_bot_precision,rank_test_bot_precision,split0_test_human_precision,split1_test_human_precision,split2_test_human_precision,split3_test_human_precision,split4_test_human_precision,split5_test_human_precision,split6_test_human_precision,split7_test_human_precision,split8_test_human_precision,split9_test_human_precision,mean_test_human_precision,std_test_human_precision,rank_test_human_precision,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,split5_test_precision,split6_test_precision,split7_test_precision,split8_test_precision,split9_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_bot_recall,split1_test_bot_recall,split2_test_bot_recall,split3_test_bot_recall,split4_test_bot_recall,split5_test_bot_recall,split6_test_bot_recall,split7_test_bot_recall,split8_test_bot_recall,split9_test_bot_recall,mean_test_bot_recall,std_test_bot_recall,rank_test_bot_recall,split0_test_human_recall,split1_test_human_recall,split2_test_human_recall,split3_test_human_recall,split4_test_human_recall,split5_test_human_recall,split6_test_human_recall,split7_test_human_recall,split8_test_human_recall,split9_test_human_recall,mean_test_human_recall,std_test_human_recall,rank_test_human_recall,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,split5_test_recall,split6_test_recall,split7_test_recall,split8_test_recall,split9_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,split5_test_roc_auc,split6_test_roc_auc,split7_test_roc_auc,split8_test_roc_auc,split9_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,method,param_clf__class_weight,param_clf__criterion,param_clf__max_depth,param_clf__max_features,param_clf__n_estimators,param_clf__learning_rate,param_clf__loss,param_clf__C,param_clf__degree,param_clf__gamma,param_clf__kernel,param_clf__solver,param_clf__booster,param_clf__eval_metric,param_clf__objective,param_clf__normalize_type,param_clf__sample_type,param_clf__feature_selector,param_clf__updater
0,0.021757,0.004324,0.047904,0.008889,{},0.709091,0.693069,0.745455,0.730769,0.747664,0.666667,0.712871,0.711538,0.6

### Best models of each classifier

In [25]:
best_models_f1_weighted = (
    df_models_performance_all
    .sort_values('mean_test_f1_weighted',ascending=False)
    .round(3)
    .drop_duplicates('method')
    [['method', 'params', 'mean_test_f1_weighted', 'mean_test_bot_f1', 'mean_test_human_f1', 
      'mean_test_f1_micro', 'mean_test_bot_recall', 'mean_test_human_recall', 'mean_test_recall', 
      'mean_test_bot_precision', 'mean_test_human_precision', 'mean_test_precision', 
      'mean_test_f1_macro','mean_test_roc_auc']]
    .reset_index(drop=True)
)
with option_context('display.max_column',None,'display.max_colwidth',None):
    display(best_models_f1_weighted)

,method,params,mean_test_f1_weighted,mean_test_bot_f1,mean_test_human_f1,mean_test_f1_micro,mean_test_bot_recall,mean_test_human_recall,mean_test_recall,mean_test_bot_precision,mean_test_human_precision,mean_test_precision,mean_test_f1_macro,mean_test_roc_auc
0,GradientBoostingClassifier,"{'clf__criterion': 'friedman_mse', 'clf__learning_rate': 0.05, 'clf__loss': 'log_loss', 'clf__max_depth': 3, 'clf__max_features': 'log2', 'clf__n_estimators': 100}",0.932,0.930,0.934,0.932,0.939,0.925,0.932,0.923,0.944,0.934,0.932,0.970
1,RandomForestClassifier,"{'clf__class_weight': 'balanced', 'clf__criterion': 'log_loss', 'clf__max_depth': 6, 'clf__max_features': 'sqrt', 'clf__n_estimators': 200}",0.922,0.921,0.922,0.922,0.947,0.899,0.922,0.897,0.948,0.924,0.922,0.969
2,XGBClassifier_dart,"{'clf__booster': 'dart', 'clf__eval_metric': 'mae', 'clf__learning_rate': 0.05, 'clf__max_depth': 4, 'clf__n_estimators': 50, 'clf__normalize_type': 'tree', 'clf__objective': 'binary:logistic', 'clf__sample_type': 'weighted'}",0.919,0.917,0.920,0.919,0.935,0.903,0.919,0.900,0.938,0.920,0.919,0.967
3,XGBClassifier_tree,"{'clf__booster': 'gbtree', 'clf__eval_metric': 'error', 'clf__learning_rate': 0.05, 'clf__max_depth': 4, 'clf__n_estimators': 50, 'clf__objective': 'binary:logistic'}",0.919,0.917,0.920,0.919,0.935,0.903,0.919,0.900,0.938,0.920,0.919,0.967
4,DecisionTreeClassifier,"{'clf__class_weight': 'balanced', 'clf__criterion': 'gini', 'clf__max_depth': 4}",0.910,0.909,0.911,0.910,0.931,0.891,0.910,0.891,0.933,0.913,0.910,0.924
5,LinearDiscriminantAnalysis,{'clf__solver': 'lsqr'},0.895,0.893,0.897,0.895,0.916,0.876,0.895,0.874,0.921,0.898,0.895,0.961
6,XGBClassifier_linear,"{'clf__booster': 'gblinear', 'clf__feature_selector': 'shuffle', 'clf__updater': 'shotgun'}",0.894,0.889,0.898,0.894,0.885,0.901,0.894,0.894,0.896,0.895,0.894,0.957
7,SVC,"{'clf__C': 0.5, 'clf__class_weight': 'balanced', 'clf__degree': 3, 'clf__gamma': 'auto', 'clf__kernel': 'poly'}",0.820,0.812,0.827,0.820,0.806,0.833,0.820,0.820,0.823,0.822,0.820,0.833
8,GaussianNB,{},0.753,0.705,0.798,0.760,0.597,0.912,0.760,0.865,0.710,0.785,0.751,0.893
